<a href="https://colab.research.google.com/github/Tomalison/hello-colab/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C_08_%E7%88%AC%E8%9F%B2%E8%B3%87%E6%96%99%E8%92%90%E9%9B%86%E8%88%87%E8%B3%87%E6%96%99%E5%BA%ABAPI%E4%B8%B2%E6%8E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json

In [2]:
# 利用 requests 對 API 來源傳送一個請求
response = requests.get('https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json')

# 將請求回應的內容存成一個字串格式
response_text = response.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(response_text)

print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
med_count = {}

# 填入欄位名稱
for d in data['features']:
    county = d['properties']['county']
    if county not in med_count:
        med_count[county] = 0
    med_count[county] += 1

print(med_count)

In [ ]:
mask_count = {}

# 填入欄位名稱
for d in data['features']:
    county = d['properties']['county']
    adult_mask_count = d['properties']['mask_adult']
    if county not in mask_count:
        mask_count[county] = 0
    mask_count[county] += adult_mask_count

# 將結果從大到小排列
mask_count = dict(sorted(mask_count.items(), key=lambda item: item[1], reverse=True))

print(mask_count)

In [ ]:
import sqlite3
import datetime

In [ ]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

# 利用 requests 對 API 來源傳送一個請求
response = requests.get('https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json')

# 將請求回應的內容存成一個字串格式
response_text = response.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(response_text)['features']

med_counts = {}
# 填入欄位名稱
for d in data:

    countys = d['properties']['county']
    if countys not in med_counts:
        med_counts[countys] = 0
    med_counts[countys] += 1

for city, counts in med_counts.items():
    t = datetime.datetime.now()
    c.execute(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
    conn.commit()

# 查詢資料
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS masks
             (city text, counts integer, createdAt datetime)''')
c.execute('''DELETE FROM masks''')
conn.commit()

# 新增資料
response = requests.get('https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json')
d = response.text
data = json.loads(d)

#計算出每個地區的成人剩餘口罩數量
mask_count = {}
# 填入欄位名稱
for d in data['features']:
  county = d['properties']['county']
  if county not in mask_count: #判斷城市是否有重複加入
      mask_count[county] = 0
  if county in mask_count:
      mask_count[county] += d['properties']['mask_adult'] #計算成人口罩數量

for county, count in mask_count.items():
    t = datetime.datetime.now()
    c.execute(f"INSERT INTO masks VALUES ('{county}', {count}, '{t}')")
    conn.commit()

# 查詢資料
c.execute("SELECT * FROM masks")
print(c.fetchall())
conn.commit()
conn.close()